# Granularity in mixed-integer nonlinear optimization

## Summary of the document

This document is complementary to the paper "Granularity in mixed-integer nonlinear optimization". The intention is to make the numerical results transparent. The generated results for the IPCP, the feasibility pump as well as for B-Hyb are stored as pandas data-frames and are concatenated herein. 

The latex-tables of the article are produced by using the command print(dataframe.to_latex(float_format = '%.2f'))

In [1]:
import pandas as pd
pd.options.display.float_format = '${:,.2f}'.format
import numpy as np
import pickle
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

## First quick check

In [5]:
result = load_obj('intermediate_results1')

In [6]:
result

,vars,constrs,time L,time SOR,obj,constr_value
autocorr_bern20_03,"(21, 20, 20)","(1, 1, 1, 1)",$0.31,$0.09,$-3.99,$-60.01
autocorr_bern25_03,"(26, 25, 25)","(1, 1, 1, 1)",$0.11,$0.08,$-29.98,$-58.02
blend029,"(102, 36, 36)","(213, 12, 0, 0)",$0.00,$0.20,$inf,$-inf
blend146,"(222, 87, 87)","(624, 24, 0, 0)",$0.00,$0.67,$inf,$-inf
blend480,"(312, 124, 124)","(884, 32, 0, 0)",$0.00,$15.39,$inf,$-inf
blend531,"(272, 104, 104)","(736, 32, 0, 0)",$0.00,$0.98,$inf,$-inf
blend718,"(222, 87, 87)","(606, 24, 0, 0)",$0.00,$0.44,$inf,$-inf
blend721,"(222, 87, 87)","(627, 24, 0, 0)",$0.00,$1.28,$inf,$-inf
blend852,"(304, 120, 120)","(860, 32, 0, 0)",$0.00,$1.06,$inf,$-inf
crudeoil_lee1_05,"(535, 40, 40)","(1240, 160, 0, 0)",$0.00,$1.22,$inf,$-inf
